In [1]:
#get_ipython().system('jupyter nbconvert --to script 1-Nui_Par_Fitting.ipynb')
import pyximport

pyximport.install()
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
import gammapy

# from gammapy.datasets import MapDataset
from gammapy.maps import Map
from astropy.coordinates import SkyCoord, Angle
from gammapy.modeling import Fit,  Parameters, Covariance , Parameter
from gammapy.datasets import MapDataset ,Datasets#, MapDatasetNuisance
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    create_crab_spectral_model,
    SkyModel,
    PointSpatialModel,
    ShellSpatialModel,
    GeneralizedGaussianSpatialModel,
    TemplateSpatialModel,
    LogParabolaSpectralModel,
    GaussianSpatialModel,
    DiskSpatialModel,
    PowerLawNormSpectralModel,
    Models,
    SpatialModel,
    FoVBackgroundModel,
    #PowerLawNuisanceSpectralModel,
    #PowerLawNormNuisanceSpectralModel
)
from gammapy.estimators import TSMapEstimator, ExcessMapEstimator

from regions import CircleSkyRegion, RectangleSkyRegion
import yaml
import sys
sys.path.append('/home/katrin/Documents/nuisance_summary/')
sys.path.append('../')
from Dataset_Creation import sys_dataset
import Dataset_load 

In [2]:

scaled_amplitude = Parameter('amplitude', 
                             value = 1e-12)
dataset_asimov  = Dataset_load.create_asimov(model = 'pl', source = "PKSflare", 
                                            parameters = Parameters([scaled_amplitude]))

loaded dataset:
/home/wecapstor1/caph/mppi045h/nuisance_summary/PKS_flare/HESS_public/dataset-simulated-pl.fits.gz


## Asimov Cases

In [3]:

shift_cor = 0.1
tilt_cor = 0.02

bias, resolution = 0,0


In [4]:
sys_d_cor = sys_dataset(dataset_asimov= dataset_asimov,
                        shift = shift_cor, 
                        tilt = tilt_cor,
                        bias = bias,
                        resolution = resolution,
                        rnd = False,
                        e_reco_creation=10)
sys_d_cor_rnd = sys_dataset(dataset_asimov= dataset_asimov,
                        shift = shift_cor, 
                        tilt = tilt_cor,
                        bias = bias,
                        resolution = resolution,
                        rnd = True,
                        e_reco_creation=10)
dataset_cor_N = sys_d_cor.create_dataset_N(10)
dataset_cor = sys_d_cor_rnd.create_dataset()




zero = 1e-24
## addional parameter bias and resolution (ereco) but are frozen
penalising_invcovmatrix = np.zeros((4, 4))
np.fill_diagonal(penalising_invcovmatrix,
                 [1/zero**2, 1/zero**2, 1/shift_cor**2, 1/tilt_cor**2])

dataset_cor_N.penalising_invcovmatrix = penalising_invcovmatrix

dataset_cor_N.irf_model.e_reco_model.parameters.freeze_all()


In [5]:
%%time
fit_cor = Fit(store_trace=False)
result_cor = fit_cor.run(dataset_cor_N)



CPU times: user 1min 51s, sys: 2min 41s, total: 4min 32s
Wall time: 1min 17s


In [6]:
%%time
fit_cor = Fit(store_trace=False)
result_cor = fit_cor.run(dataset_cor)



CPU times: user 12.7 s, sys: 2.72 s, total: 15.4 s
Wall time: 15.5 s


In [7]:
#CPU time:
(15.8 *u.s * 1000 ).to('hr')

<Quantity 4.38888889 h>

In [8]:
(15.8 *u.s * 100 ).to('hr')

<Quantity 0.43888889 h>

In [9]:
(4*u.min +  32*u.s).to('hr')

<Quantity 0.07555556 h>